# Пример модели, обученной на верно размеченных данных

## Установка и импорт необходимых библиотек 

In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 8.7 MB/s 
     |████████████████████████████████| 1.2 MB 48.5 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.1/457.1 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.6/660.6 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.2.0
    Uninstalling typing_extensions-4.2.0:
      Successfully uninstalled typing_extensions-4.2.0
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    Uninstalling smart-open-6.0.0:
      Successfully uni

In [2]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.3.1                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines        en_core_web_trf (3.3.0)       



In [3]:
import spacy
from spacy.tokens import DocBin
from pathlib import Path


## Подготовка данных

In [4]:
nlp = spacy.blank("en")

In [5]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe('ner')

In [6]:
LABELS  =['PERSON','ORG']
for ent in LABELS:
    ner.add_label(ent)

In [39]:
'I like R-Vision and Microsoft.'[20:29]

'Microsoft'

In [41]:
TRAIN_DATA = [
    ('Who is Nastya?',[(7, 13, 'PERSON')]),

     ('Who is Vitalik?',[(7, 14, 'PERSON')]),

    ('I like R-Vision and Microsoft.', [(7, 15, 'ORG'), (20, 29, 'ORG')])
]

db = DocBin()
for text, annotations in TRAIN_DATA:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        ents.append(span)

    doc.ents = ents
    db.add(doc)

db.to_disk("./train.spacy")
db.to_disk("./dev.spacy")

## Обучение модели

In [42]:
!python -m spacy init config --lang en --pipeline ner /content/config.cfg --force

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
/content/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [43]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output /content/ner_demo/training/ --training.max_steps 100 --training.eval_frequency 10

✔ Created output directory: /content/ner_demo/training
ℹ Saving to output directory: /content/ner_demo/training
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-06-11 17:06:37,964] [INFO] Set up nlp object from config
[2022-06-11 17:06:37,977] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-06-11 17:06:37,982] [INFO] Created vocabulary
[2022-06-11 17:06:37,984] [INFO] Finished initializing nlp object
[2022-06-11 17:06:38,134] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     11.40    0.00    0.00    0.00    0.00
 10      10          0.17    103.55    0.00    0.00    0.00    0.00
 20      20          1.27     39.10 

## Тестирование качества обученной модели

In [46]:
nlp_loaded = spacy.load("/content/ner_demo/training/model-best")

In [48]:
doc = nlp_loaded("Vitalik work at Microsoft") 
spacy.displacy.render(doc, style="ent", jupyter=True) 


In [51]:
doc = nlp_loaded("Dad want to call her rat Nastya") 
spacy.displacy.render(doc, style="ent", jupyter=True) 

In [54]:
[(str(x), x.label_) for x in doc.ents]

[('Nastya', 'PERSON')]